In [1]:
import pandas as pd
import numpy as np
import datetime
import warnings

In [2]:
pd.options.mode.chained_assignment = None
pd.set_option('display.max_columns', None)
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [3]:
def delete_duplicates(data,param_combined,param1,param2):
    data[param_combined] = data[param1].astype(str) + ' ' + data[param2].astype(str)
    data = data.drop_duplicates(subset=param_combined)
    data = data.drop(columns=param_combined)
    return data
def merge_left(left_data,right_data,param_combined,param1,param2,left_columns,right_columns):
    left_data[param_combined] = left_data[param1].astype(str) + ' ' + left_data[param2].astype(str)
    right_data[param_combined] = right_data[param1].astype(str) + ' ' + right_data[param2].astype(str)
    left_columns.append(param_combined)
    left_columns = list(set(left_columns))
    right_columns.append(param_combined)
    right_columns = list(set(right_columns))
    output = left_data[left_columns].merge(right_data[right_columns],on=param_combined,how='left')
    left_data = left_data.drop(columns=param_combined)
    right_data = right_data.drop(columns=param_combined)
    output = output.drop(columns=param_combined)
    return output

PREPROCESSING
(SKIP JIKA SUDAH RUN DATABASE bond_vt !!)

In [4]:
bond_identity = pd.read_csv('bond_msbond.csv')
bond_price_1 = pd.read_csv('Bond_TrBei_to_20240816.csv')
bond_price_2 = pd.read_csv('Bond_TrBei_from_20240819_to_20250226.csv')
bond_price = pd.concat([bond_price_1,bond_price_2])
bond = bond_price.merge(bond_identity,on='portId',how='left')
bond['issuedyear'] = bond['issueddate'].str[:4]
bond['issuedyear'] = bond['issuedyear'].astype('int')
bond = bond.drop(columns='txtNo')
bond = delete_duplicates(data=bond,param_combined='portId + issueddate',param1='portId',param2='issueddate')

/var/folders/rj/b4yv8pv54v31_6jpqn2byxgc0000gn/T/ipykernel_6431/2532939075.py:2: DtypeWarning: Columns (13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  bond_price_1 = pd.read_csv('Bond_TrBei_to_20240816.csv')


In [5]:
bond['portReleased'] = bond['portReleased'].replace(r'\N',np.nan)
bond['jtdatedt'] = pd.to_datetime(bond['jtdate'])
bond['portReleaseddt'] = pd.to_datetime(bond['portReleased'])
bond['tenorday'] = bond['jtdatedt'] - bond['portReleaseddt']
bond['tenorday'] = pd.to_numeric(bond['tenorday'].astype('str').str[:-5],errors='coerce')
bond['tenoryear'] = bond['tenorday']/365
bond = bond.drop(columns=['jtdatedt','portReleaseddt'])
bond = delete_duplicates(data=bond,param_combined='portId + issueddate',param1='portId',param2='issueddate')

In [ ]:
cols = list(bond.columns)
bond['tradeday'] = 'Y'
bond_i_list = []
for i in bond['portId'].unique():
    bond_i = bond[bond['portId']==i]
    start_date = bond_i['portReleased'].unique()[0]
    end_date = max(bond['issueddate'].unique())
    jt_date = bond_i['jtdate'].unique()[0]
    if type(jt_date) is str:
        if end_date>jt_date:
            end_date = jt_date
        else:
            pass
    else:
        print(f'JT date for {i} is not found')
        continue
    if type(start_date) is str:
        bond_i_date = pd.DataFrame({'issueddate':pd.date_range(start=start_date,end=end_date,freq='D')})
        if len(bond_i_date)>0:
            bond_i_date['issueddate'] = bond_i_date['issueddate'].astype('str')
            bond_i = bond_i_date.merge(bond_i,on='issueddate',how='left')
            bond_i = bond_i.sort_values(by='issueddate')
            bond_i.loc[:,cols] = bond_i.loc[:,cols].ffill()
            bond_i_list.append(bond_i)
        else:
            bond_i_list.append(bond_i)
    else:
        print(f'Released date for {i} is not found')
        continue
bond = pd.concat(bond_i_list)
bond = bond.dropna(subset='portId')
bond = delete_duplicates(data=bond,param_combined='portId + issueddate',param1='portId',param2='issueddate')
bond = bond.reset_index(drop=True)
bond['tradeday'] = bond['tradeday'].fillna('N')
bond['weekend'] = np.where((pd.to_datetime(bond['issueddate'], errors='coerce')).dt.dayofweek<5,'N','Y')



In [7]:
#Tambah kurs BI
kurs_BI = pd.read_excel('kurs_BI.xls')
kurs_BI.columns = kurs_BI.loc[0]
kurs_BI = kurs_BI[1:].reset_index(drop=True)
kurs_BI = kurs_BI.rename(columns={'Nilai': 'USDIDR', 'Tanggal': 'issueddate'})
kurs_BI['issueddate'] = kurs_BI['issueddate'].apply(lambda x: x.strftime('%Y-%m-%d'))
bond = pd.merge(bond, kurs_BI[['issueddate', 'USDIDR']], on='issueddate', how='left')
bond_list = []
for i in bond['portId'].unique():
    bond_i = bond[bond['portId']==i]
    bond_i = bond_i.sort_values(by='issueddate')
    bond_i['USDIDR'] = bond_i['USDIDR'].ffill()
    bond_list.append(bond_i)
bond = pd.concat(bond_list)
bond = delete_duplicates(data=bond,param_combined='portId + issueddate',param1='portId',param2='issueddate')
bond = bond.reset_index(drop=True)
bond['outstanding'] = np.where(bond['dollar']=='Y',bond['outstanding']*bond['USDIDR'],bond['outstanding'])

In [8]:
#Tambah nominal
bond_nominal = pd.read_csv('bond_msnominal.csv').drop('txtno',axis=1).rename(columns={'portid': 'portId','tanggal': 'issueddate'})
bond_nominal['portId + issueddate'] = bond_nominal['portId'].astype(str) + ' ' + bond_nominal['issueddate'].astype(str)
bond['portId + issueddate'] = bond['portId'].astype(str) + ' ' + bond['issueddate'].astype(str)
bond = bond.merge(bond_nominal[['portId + issueddate','nominal']],how='left',on='portId + issueddate')
bond = bond.drop(columns='portId + issueddate')
bond_list = []
for i in bond['portId'].unique():
    bond_i = bond[bond['portId']==i]
    bond_i = bond_i.sort_values(by='issueddate')
    outstanding_i = bond_i['outstanding'].dropna().unique()[0]
    bond_i['nominal'] = bond_i['nominal'].ffill()
    bond_i['nominal'] = bond_i['nominal'].fillna(outstanding_i)
    bond_list.append(bond_i)
bond = pd.concat(bond_list)
bond = delete_duplicates(data=bond,param_combined='portId + issueddate',param1='portId',param2='issueddate')
bond = bond.reset_index(drop=True)



In [9]:
#Tambah accrued interest
bond_list = []
for i in bond['portId'].unique():
    bond_i = bond[bond['portId']==i]
    coupon_i = bond_i['coupon'].unique()[0]*0.01
    outstanding_i = bond_i['outstanding'].unique()[0]
    bond_i['interest'] = (coupon_i*outstanding_i)/365
    bond_i = bond_i.sort_values(by='issueddate')
    bond_i['accruedinterest'] = bond_i['interest'].cumsum()
    bond_list.append(bond_i)
bond = pd.concat(bond_list)
bond = delete_duplicates(data=bond,param_combined='portId + issueddate',param1='portId',param2='issueddate')
bond = bond.reset_index(drop=True)
bond['marketvalue'] = bond['nominal']*bond['lastPrice']*0.01 + bond['accruedinterest']


In [10]:
sbn = bond[bond['portType']=='S']
corpo = bond[bond['portType']=='B']

In [11]:
sbn_bday = sbn[sbn['weekend']=='N']

In [ ]:
#Calculate average and sum transaction value over 3 months
period = 21*3 #days
date_list = [i.strftime('%Y-%m-%d') for i in pd.bdate_range(min(bond['issueddate']),max(bond['issueddate']))]
sbn_j_list = []
for en,i in enumerate(set(list(sbn_bday['portId']))):
    sbn_i = sbn_bday[sbn_bday['portId']==i].drop_duplicates()
    sbn_i = sbn_i.sort_values(by='issueddate')
    sbn_i['issueddatedt'] = pd.to_datetime(sbn_i['issueddate'])
    for j in list(sbn_i['issueddate'].unique()):
        sbn_j = sbn_i[sbn_i['issueddate']==j]
        try:
            pd_pos = date_list.index(j) - period #previous date index position
            cd_ = datetime.datetime.strptime(j, '%Y-%m-%d')
            pd_ = datetime.datetime.strptime(date_list[pd_pos], '%Y-%m-%d')
            sbn_i_inside_dates = sbn_i[(sbn_i['issueddatedt']>=pd_)&(sbn_i['issueddatedt']<=cd_)]
            if len(sbn_i_inside_dates)>0:
                sbn_j[f'sumvalueTrans_{period}'] = sum(sbn_i_inside_dates['valueTrans'])
                sbn_j[f'avgvalueTrans_{period}'] = sum(sbn_i_inside_dates['valueTrans'])/len(sbn_i_inside_dates['valueTrans'])
                sbn_j[f'pctdayvalueTrans_{period}'] = len(sbn_i_inside_dates['valueTrans'])/(period+1)
                sbn_j_list.append(sbn_j)
            else: pass
        except ValueError: pass
sbn_vt_bday = pd.concat(sbn_j_list)
sbn_vt_bday = delete_duplicates(data=sbn_vt_bday,param_combined='portId + issueddate',param1='portId',param2='issueddate')
sbn_vt_bday = sbn_vt_bday.reset_index(drop=True)
sbn_vt_bday = sbn_vt_bday.drop(columns='issueddatedt')
sbn_vt = merge_left(left_data=sbn,right_data=sbn_vt_bday,param_combined='portId + issueddate',param1='portId',param2='issueddate',left_columns=list(sbn.columns),right_columns=['sumvalueTrans_63','avgvalueTrans_63','pctdayvalueTrans_63'])
sbn_vt_list = []
for i in set(list(sbn_vt['portId'])):
    sbn_vt_i = sbn_vt[sbn_vt['portId']==i]
    sbn_vt_i = sbn_vt_i.sort_values(by='issueddate')
    sbn_vt_i[['sumvalueTrans_63', 'pctdayvalueTrans_63', 'avgvalueTrans_63']] = sbn_vt_i[['sumvalueTrans_63', 'pctdayvalueTrans_63', 'avgvalueTrans_63']].ffill()
    sbn_vt_list.append(sbn_vt_i)
sbn_vt = pd.concat(sbn_vt_list)
sbn_vt = sbn_vt.reset_index(drop=True)


In [ ]:
#sbn_vt.to_csv('bond_database_preprocessed.csv')

INDEXING

In [ ]:
sbn_vt = pd.read_csv('bond_database_preprocessed.csv')

In [ ]:
#Generate portfolio composition
data = sbn_vt.copy()
start_nab = 10000
reb_date = ['2020-01-01','2020-04-01','2020-07-01','2020-10-01'
            ,'2021-01-01','2021-04-01','2021-07-01','2021-10-01'
            ,'2022-01-01','2022-04-01','2022-07-01','2022-10-01'
            ,'2023-01-01','2023-04-01','2023-07-01','2023-10-01'
            ,'2024-01-01','2024-04-01','2024-07-01','2024-10-01','2025-02-03'] #Harus Urut dari terkecil ke terbesar! Rebalancing date terakhir tidak dihitung!
sort_param = 'sumvalueTrans_63'
sort_by = 'High' #'High' untuk top nilai terbesar, 'Low' untuk top nilai terkecil
num_assets = 50
screening_param = ['pctdayvalueTrans_63']
screening_sign = ['Above'] #'Above' or 'Below'
screening_val = [0.9]
basic_param = ['issueddate','portId','portName','lastPrice','outstanding','coupon','portReleased','jtdate',sort_param]
drop_name_list = [] #Drop nama bond tertentu
drop_type_list = ['VR','SPN'] #Drop tipe Bond dari string


basic_param = list(set(basic_param))
extend_param = basic_param.copy()
extend_param.extend(screening_param)
extend_param = list(set(extend_param))
database_part = sbn_vt[extend_param]
database_date_list = list(sbn_vt['issueddate'].sort_values().unique())
database_date_list_ = [d for d in database_date_list if d >= reb_date[0] and d <= reb_date[-1]]
buy_date_list = reb_date[:-1]
sell_date_list = []
for i in reb_date[1:]:
    sell_date_i = database_date_list_.index(i)-1
    sell_date = database_date_list_[sell_date_i]
    sell_date_list.append(sell_date)
param_date_list = []
for i in buy_date_list:
    param_date_i = database_date_list.index(i)-1
    param_date = database_date_list[param_date_i]
    param_date_list.append(param_date)
database_i_list = []
for en,i in enumerate(param_date_list):
    database_i = database_part[database_part['issueddate']==i]
    database_i = database_i.dropna(subset=['issueddate', 'portId', sort_param] + [f'{s}' for s in screening_param])
    if len(drop_type_list)>0:
        for t in drop_type_list:
            database_i = database_i[~database_i['portId'].str.contains(t)]
    else:
        pass
    if len(drop_name_list)>0:
        for r in drop_name_list:
            database_i = database_i[database_i['portId']!=n]
    else:
        pass
    if len(screening_param) == len(screening_sign) == len(screening_val):
        for ens,j in enumerate(screening_param):  #Initial Screening
            if screening_sign[ens] == 'Above':
                database_i = database_i[database_i[j]>screening_val[ens]]
            elif screening_sign[ens] == 'Below':
                database_i = database_i[database_i[j]<screening_val[ens]]
            else:
                raise TypeError('screening_sign wrong. Select either Above or Below')
    else:
        raise Exception('Number of elements in screening parameter list are different')
    database_i = database_i[basic_param] 
    if sort_by=='High':
        database_i = database_i.sort_values(by=sort_param,ascending=False)
        database_i = database_i.head(num_assets)
        database_i['rank'] = database_i[sort_param].rank()
    elif sort_by=='Low':
        database_i = database_i.sort_values(by=sort_param,ascending=True)
        database_i = database_i.head(num_assets)
        database_i['rank'] = database_i[sort_param].rank(ascending=False)
    else:
        raise TypeError('sort_param wrong. Check your typing')
    if len(database_i) == num_assets:
        pass
    else:
        raise Exception('Number of stocks in one portfolio is less than specified')
    database_i['from'] = buy_date_list[en]
    database_i['to'] = sell_date_list[en]
    database_i['number'] = en
    database_i_list.append(database_i)
reb_comp_ = pd.concat(database_i_list)
reb_comp_ = reb_comp_.reset_index(drop=True)
reb_comp = reb_comp_[['portId','from','to','number']]

In [ ]:
#Generate nama bond di dalam portfolio dan date-nya
num_and_date_list = []
for i in reb_comp['number'].unique():
    reb_comp_i = reb_comp[reb_comp['number']==i]
    start_date = reb_comp_i['from'].unique()[0]
    end_date = reb_comp_i['to'].unique()[0]
    dates = [d.strftime('%Y-%m-%d') for d in pd.date_range(start_date,end_date)]
    num_and_date = pd.DataFrame({'issueddate':dates})
    num_and_date['number'] = i
    num_and_date_list.append(num_and_date)
num_and_dates = pd.concat( num_and_date_list)
num_and_dates = num_and_dates.sort_values(by='issueddate')

In [ ]:
#Indexing. Generate Excel file
data = sbn_vt.copy()
data_list = []
data_i_list = []
with pd.ExcelWriter(f'Bond Index.xlsx') as writer:
    for en,i in enumerate(num_and_dates['issueddate']):
        data_i = data[data['issueddate']==i].drop_duplicates(subset='portId')
        num = num_and_dates[num_and_dates['issueddate']==i]['number'].values[0]
        bonds = reb_comp[reb_comp['number']==num][['portId']]
        data_i = bonds.merge(data_i,on='portId',how='left')
        data_i = data_i.dropna(subset='marketvalue') 
        total_nominal= data_i['nominal'].sum()
        total_interest = data_i['interest'].sum()
        total_marketvalue = data_i['marketvalue'].sum()
        if en<1:
            total_accruedinterest = data_i['accruedinterest'].sum()
            total_unit = total_marketvalue/start_nab
            nab = total_marketvalue/total_unit
        else:
            data_i_t_1 = data_list[en-1]
            total_accruedinterest = total_accruedinterest+total_interest
            nab = (total_marketvalue-(total_nominal-data_i_t_1['Total Nominal'].values[0]))/total_unit
            total_unit = total_unit+(total_nominal-data_i_t_1['Total Nominal'].values[0])/nab
        df = pd.DataFrame({'Date':i,'Day':[pd.to_datetime(i).day_name()],'Total Nominal':[total_nominal],'Total Interest':[total_interest],'Total Accrued Interest':[total_accruedinterest],
                                'Total Market Value':[total_marketvalue],'Total Unit':[total_unit],'NAB':[nab]},index=[en])
        data_list.append(df)
        data_i_list.append(data_i)
    bond_index = pd.concat(data_list)
    detail = pd.concat(data_i_list)
    bond_index.to_excel(writer, sheet_name='NAB')
    reb_comp.to_excel(writer, sheet_name='Composition')
    detail.to_excel(writer, sheet_name='Detail')
